Si queremos integrar en un intervalo distinto de $(0,1)$, el truco de Monte Carlo sigue funcionando, pero hay que **reformular la variable aleatoria y su distribución**:

---

### 1️⃣ Transformación de intervalo

Si la integral es:

$$
\theta = \int_a^b g(x) \, dx
$$

podemos usar un cambio de variable

$$
x = a + (b-a)u, \quad u \sim \text{Uniform}(0,1)
$$

y $dx = (b-a) du$, lo que lleva a:

$$
\theta = (b-a) \, \mathbb{E}[g(a + (b-a)U)]
$$

donde $U$ es uniforme en $(0,1)$.

---

### 2️⃣ Interpretación Monte Carlo

Generas $U_1, \dots, U_n \sim \text{Uniform}(0,1)$ y calculas:

$$
\hat{\theta}_n = (b-a) \cdot \frac{1}{n} \sum_{i=1}^n g(a + (b-a) U_i)
$$

La LGN asegura que $\hat{\theta}_n \to \theta$.

---

### 3️⃣ Ejemplo numérico

Supongamos:

$$
\theta = \int_{2}^{5} \sin(x) \, dx
$$

Transformamos:

$$
x = 2 + 3u, \quad dx = 3\,du
$$

Entonces:

$$
\theta \approx 3 \cdot \frac{1}{n} \sum_{i=1}^n \sin(2 + 3 U_i)
$$

---

Si quieres, puedo hacerte el **Plotly interactivo adaptado para $(a,b) \neq (0,1)$**, así verías cómo funciona Monte Carlo y la LGN en cualquier intervalo. ¿Quieres que te lo prepare?


In [4]:
import numpy as np
import math
import matplotlib
matplotlib.use("TkAgg")  # Cambia a "Qt5Agg" si tienes PyQt5
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button

# --------- CONFIGURACIÓN ---------
a, b = 2, 5
g = np.sin
N_MAX = 5000

# Valor verdadero
theta_true = -math.cos(b) + math.cos(a)

# Simulación Monte Carlo
rng = np.random.default_rng(20250812)
u = rng.random(N_MAX)
x_vals = a + (b - a) * u
g_vals = g(x_vals)
cum_mean = (b - a) * np.cumsum(g_vals) / np.arange(1, N_MAX + 1)

# --------- PLOTEO ---------
fig, ax = plt.subplots()
plt.subplots_adjust(bottom=0.25)
l_est, = ax.plot(np.arange(1, 101), cum_mean[:100], lw=2)
ax.axhline(y=theta_true, color='r', linestyle='--', label=f"Valor real = {theta_true:.6f}")
ax.set_xlabel("Número de muestras (n)")
ax.set_ylabel("Estimación de la integral")
ax.set_title("Estimación de la integral de g(x) = sin(x) en [2, 5] usando Monte Carlo")
ax.legend()

# Slider para cambiar n
ax_slider = plt.axes([0.25, 0.1, 0.5, 0.03])
slider_n = Slider(ax_slider, 'n', 10, N_MAX, valinit=100, valstep=10)

def update(val):
    n = int(slider_n.val)
    l_est.set_xdata(np.arange(1, n + 1))
    l_est.set_ydata(cum_mean[:n])
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw_idle()

slider_n.on_changed(update)

# Botón para regenerar muestra
ax_button = plt.axes([0.8, 0.025, 0.1, 0.04])
btn = Button(ax_button, 'Resample')

def resample(event):
    global u, x_vals, g_vals, cum_mean
    u = rng.random(N_MAX)
    x_vals = a + (b - a) * u
    g_vals = g(x_vals)
    cum_mean = (b - a) * np.cumsum(g_vals) / np.arange(1, N_MAX + 1)
    update(slider_n.val)

btn.on_clicked(resample)

plt.show()
